# Exploring the business potential for opening a hotel in Madrid

## 1.Introduction

###### This report explores the opportunities for the opening of a new hotel in the city of Madrid.
Madrid is the European tourist capital with the most average expenditure per tourist with € 355 per day.

During the last 6 years, the number of foreign tourists who have visited the Community of Madrid maintains a sustained growth, well above the rest of Spanish destinations.

The city of Madrid recorded in 2019 the arrival of 10.4 million visitors (+2.1% vs 2018), of which 55% were international and 45% national. 22.6 million overnight stays were generated (+5.7% vs 2018). The main international tourism source markets were the United States, Italy, France, the United Kingdom and Germany, and the fastest growing markets were the United States, China, Italy, Brazil and Mexico. A total expense of 10,451 million was reached (+12.6% vs 2018) and an average daily expense of 270 euros (+11% vs 2018).

The number of rooms in 5-star hotels in Madrid is growing, although it is still below that of other European capitals. This represents a business opportunity in a growth environment.

In the capital there are more than 5,100 rooms available in 5-star establishments compared to almost 22,000 in Paris or more than 18,000 in London.  Most of the hotels in Madrid are 4-stars, therefore this is the segment with the highest competition.


## 2. The data

###### The following data sets will be used for this report:
- 	A dataset containing the districts and neighborhoods of Madrid will be used. This dataset will include the geographic coordinates of each district for subsequent location on maps and clusters. Madrid has 21 districts that, in addition, group a total of 131 administrative neighborhoods. This information will be obtained from Wikipedia (https://en.wikipedia.org/wiki/Districts_of_Madrid) and from official organizations such as (http://gestiona.madrid.org/iestadis/) and (http://gestiona.madrid.org/nomecalles/DescargaBDTCorte.icm#delimitacionesterritoriales)

- 	We will use Foursquare.com to obtain leisure venues that may be of interest to tourists, from each district and existing hotels. Crossing this information with the previous one of the districts we will be able to group the information obtained in clusters and we will be able to represent them on maps for a better understanding. In this way we will be able to determine the best location for the new hotel.

###### Scrapping Madrid Districts Table from Wikipedia

In [1]:
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 

URL ='https://en.wikipedia.org/wiki/Districts_of_Madrid'
res = requests.get(URL) 
scraped_data = BeautifulSoup(res.text, 'html.parser')


In [2]:
scraped_data



<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Districts of Madrid - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpuNMQpAML8AAB1vUC4AAAAU","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Districts_of_Madrid","wgTitle":"Districts of Madrid","wgCurRevisionId":932631329,"wgRevisionId":932631329,"wgArticleId":10147562,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Spanish-language sources (es)","Articles with short description","Articles that may contain original research from March 2014","All articles that may contai

###### Cleaning scraped_data

In [3]:
data_rows = scraped_data.select('.wikitable tr')

In [4]:
data_header = data_rows[0]
data_content = data_rows[1:] 

In [5]:
data_columns = []

for column in data_header.select('th'): 
    column_name = column.getText()
    data_columns.append(column_name.replace('\n', ''))  
    
data_columns

['District Number',
 'Name',
 'District area[n 1] (Ha.)',
 'Population',
 'Population density(Hab./Ha.)',
 'Location',
 'Administrative wards']

In [6]:
final_data = []

for row in data_content:
    data_row = []
    for data in row.select('td'): 
        data_row.append(data.getText().replace('\n', ''))
    final_data.append(data_row)

###### Creating Pandas DataFrame with districts

In [26]:
MadridDF = pd.DataFrame(data=final_data, columns=data_columns)
MadridDF.head()

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Location,Administrative wards
0,1,Centro,522.82,"131,928",252.34,,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,2,Arganzuela,646.22,"151,965",235.16,,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,3,Retiro,546.62,"118,516",216.82,,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (...
3,4,Salamanca,539.24,"143,800",266.67,,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,5,Chamartín,917.55,"143,424",156.31,,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...


In [27]:
MadridDF.shape

(22, 7)

In [28]:
MadridDF.tail()

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Location,Administrative wards
17,18,Villa de Vallecas,"5,146.72","104,421",19.86,,Casco Histórico de Vallecas (181)Santa Eugenia...
18,19,Vicálvaro,"3,526.67","70,051",19.86,,Casco Histórico de Vicálvaro (191)Valdebernard...
19,20,San Blas-Canillejas,"2,229.24","154,357",69.24,,Simancas (201)Hellín (202)Amposta (203)Arcos (...
20,21,Barajas,"4,192.28","46,876",11.18,,Alameda de Osuna (211)Aeropuerto (212)Casco Hi...
21,None,None,None,None,None,None,None


In [29]:
MadridDF = MadridDF.drop(['Location', 'Administrative wards'], axis=1)

In [31]:
MadridDF = MadridDF.drop([21], axis=0)
MadridDF.tail()

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.)
16,17,Villaverde,"2,018.76","142,608",70.64
17,18,Villa de Vallecas,"5,146.72","104,421",19.86
18,19,Vicálvaro,"3,526.67","70,051",19.86
19,20,San Blas-Canillejas,"2,229.24","154,357",69.24
20,21,Barajas,"4,192.28","46,876",11.18


###### Creating Pandas Dataframe with coordinates

In [69]:
coordinates = pd.read_csv('/Users/jgonzalez/Desktop/projects/Peer-graded-Assignment-Capstone-Project---The-Battle-of-Neighborhoods-/Data/CALLEJERO_VIGENTE_DISTRITOS_201809.csv')

In [70]:
coordinates


,district Number,Latitude,Longitude
0,1,40.415347,-3.707371
1,2,40.402733,-3.695403
2,3,40.408072,-3.676729
3,4,40.429722,-3.679750
4,5,40.462059,-3.676600
5,6,40.459751,-3.697500
6,7,40.434040,-3.703790
7,8,40.478611,-3.709722
8,9,40.435151,-3.718765
9,10,40.411360,-3.708417


In [71]:
Final_MadridDF = pd.concat([MadridDF,coordinates], axis=1, sort=False)
Final_MadridDF = Final_MadridDF.drop(['district Number'], axis=1)

In [72]:
Final_MadridDF

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Latitude,Longitude
0,1,Centro,522.82,"131,928",252.34,40.415347,-3.707371
1,2,Arganzuela,646.22,"151,965",235.16,40.402733,-3.695403
2,3,Retiro,546.62,"118,516",216.82,40.408072,-3.676729
3,4,Salamanca,539.24,"143,800",266.67,40.429722,-3.679750
4,5,Chamartín,917.55,"143,424",156.31,40.462059,-3.676600
5,6,Tetuán,537.47,"153,789",286.13,40.459751,-3.697500
6,7,Chamberí,467.92,"137,401",293.64,40.434040,-3.703790
7,8,Fuencarral-El Pardo,"23,783.84","238,756",10.04,40.478611,-3.709722
8,9,Moncloa-Aravaca,"4,653.11","116,903",25.12,40.435151,-3.718765
9,10,Latina,"2,542.72","233,808",91.95,40.411360,-3.708417


###### Map of Madrid with districts superimposed on top.

In [73]:
import folium

In [74]:
# Coordinates od Madrid
latitude = 40.418889
longitude = -3.691944

# create map of New York using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(Final_MadridDF['Latitude'], Final_MadridDF['Longitude'], Final_MadridDF['Name']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

## 3.Methodology

###### Define Foursquare Credentials

In [75]:
CLIENT_ID = 'PKOBLYCFF1NEKHLDPTGWIJKRMGJENMPWPL4WEEZAKZZKL2L1' # your Foursquare ID
CLIENT_SECRET = 'M4LTSJGUGTGBPGL1RWX20F3TD3NPTSQRAZE3X5WCKDGEMZEG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PKOBLYCFF1NEKHLDPTGWIJKRMGJENMPWPL4WEEZAKZZKL2L1
CLIENT_SECRET:M4LTSJGUGTGBPGL1RWX20F3TD3NPTSQRAZE3X5WCKDGEMZEG


###### This is a function to process all the districts in Madrid.

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

###### Creating a new dataframe with Madrid venues.

In [77]:
Madrid_venues = getNearbyVenues(names=Final_MadridDF['Name'],
                                   latitudes=Final_MadridDF['Latitude'],
                                   longitudes=Final_MadridDF['Longitude']
                                    )

Centro
Arganzuela
Retiro
Salamanca
Chamartín
Tetuán
Chamberí
Fuencarral-El Pardo
Moncloa-Aravaca
Latina
Carabanchel
Usera
Puente de Vallecas
Moratalaz
Ciudad Lineal
Hortaleza
Villaverde
Villa de Vallecas
Vicálvaro
San Blas-Canillejas
Barajas


In [78]:
Madrid_venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,40.415347,-3.707371,Plaza Mayor,40.415527,-3.707506,Plaza
1,Centro,40.415347,-3.707371,Mercado de San Miguel,40.415443,-3.708943,Market
2,Centro,40.415347,-3.707371,La Taberna de Mister Pinkleton,40.414536,-3.708108,Other Nightlife
3,Centro,40.415347,-3.707371,The Hat Madrid,40.414343,-3.707120,Hotel
4,Centro,40.415347,-3.707371,Bodegas Ricla,40.414266,-3.708077,Wine Bar


###### Getting the number of venues per District

In [95]:
Number_venues_district = Madrid_venues.groupby('District').count()

In [96]:
Number_venues_district

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Arganzuela,89,89,89,89,89,89
Barajas,31,31,31,31,31,31
Carabanchel,8,8,8,8,8,8
Centro,75,75,75,75,75,75
Chamartín,24,24,24,24,24,24
Chamberí,100,100,100,100,100,100
Ciudad Lineal,31,31,31,31,31,31
Fuencarral-El Pardo,40,40,40,40,40,40
Hortaleza,19,19,19,19,19,19


###### Breakdown by categories

In [97]:
Number_venues_district_category = Madrid_venues.groupby('District')['Venue Category'].value_counts()

In [99]:
Number_venues_district_category

District    Venue Category          
Arganzuela  Restaurant                  10
            Spanish Restaurant           8
            Grocery Store                5
            Bakery                       4
            Tapas Restaurant             4
                                        ..
Villaverde  Grocery Store                4
            Gastropub                    1
            Mediterranean Restaurant     1
            Spanish Restaurant           1
            Train Station                1
Name: Venue Category, Length: 486, dtype: int64

###### Number of hotels per district

In [115]:
Madrid_Hotels = Madrid_venues[Madrid_venues['Venue Category'] == 'Hotel'] 
Madrid_Hotels.groupby('District')['Venue'].count()

District
Arganzuela            2
Barajas               7
Centro                2
Chamberí              1
Ciudad Lineal         2
Latina                6
Moncloa-Aravaca       1
Puente de Vallecas    2
Salamanca             1
Tetuán                2
Name: Venue, dtype: int64

###### Name of the hotels in each district

In [135]:
values = Madrid_Hotels.groupby('District')['Venue'].value_counts()
values

District            Venue                               
Arganzuela          AC Hotel by Marriott Atocha             1
                    AC Hotel by Marriott Carlton Madrid     1
Barajas             Hostal Aeropuerto                       1
                    Hotel Clement Barajas                   1
                    Hotel Ibis Madrid Aeropuerto Barajas    1
                    Hotel Villa de Barajas                  1
                    Meliá Barajas                           1
                    Tach Hotel Madrid                       1
                    Tryp Alameda Aeropuerto                 1
Centro              Hotel Liabeny                           1
                    The Hat Madrid                          1
Chamberí            Gran Hotel Conde Duque                  1
Ciudad Lineal       ILUNION Atrium                          1
                    hotel artiem madrid fresh people        1
Latina              Eurostars Plaza Mayor                   1
             

###### Analyze each district

In [138]:
# one hot encoding
Madrid_onehot = pd.get_dummies(Madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Madrid_onehot['District'] = Madrid_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [Madrid_onehot.columns[-1]] + list(Madrid_onehot.columns[:-1])
Madrid_onehot = Madrid_onehot[fixed_columns]

Madrid_onehot

,District,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Barajas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
788,Barajas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
789,Barajas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
790,Barajas,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###### Let's explore the mean of the frequency of occurrence of each category by district

In [143]:
Madrid_grouped = Madrid_onehot.groupby('District').mean().reset_index()
Madrid_grouped

,District,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop,Women's Store
0,Arganzuela,0.000000,0.000000,0.022472,0.000000,0.000000,0.000000,0.011236,0.000000,0.000000,...,0.011236,0.000,0.011236,0.011236,0.000,0.00,0.000,0.000000,0.000000,0.000000
1,Barajas,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.032258,0.000000,0.000000
2,Carabanchel,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000
3,Centro,0.013333,0.000000,0.000000,0.013333,0.000000,0.013333,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.026667,0.000000,0.000000
4,Chamartín,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000
5,Chamberí,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,...,0.000000,0.010,0.000000,0.000000,0.000,0.02,0.000,0.010000,0.000000,0.000000
6,Ciudad Lineal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000
7,Fuencarral-El Pardo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.025,0.000000,0.000000,0.025,0.00,0.025,0.000000,0.000000,0.000000
8,Hortaleza,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.000000
9,Latina,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.03,0.000,0.020000,0.000000,0.000000


###### Let's explore the mean of the frequency of occurrence of each hotel category by district

In [144]:
Madrid_hotels_grouped = Madrid_onehot.groupby('District')['Hotel'].mean().reset_index()
Madrid_hotels_grouped

,District,Hotel
0,Arganzuela,0.022472
1,Barajas,0.225806
2,Carabanchel,0.000000
3,Centro,0.026667
4,Chamartín,0.000000
5,Chamberí,0.010000
6,Ciudad Lineal,0.064516
7,Fuencarral-El Pardo,0.000000
8,Hortaleza,0.000000
9,Latina,0.060000


###### Showing the top 5 most common venues for each district

In [145]:
num_top_venues = 5

for district in Madrid_grouped['District']:
    print("----"+district+"----")
    temp = Madrid_grouped[Madrid_grouped['District'] == district].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arganzuela----
                venue  freq
0          Restaurant  0.11
1  Spanish Restaurant  0.09
2       Grocery Store  0.06
3    Tapas Restaurant  0.04
4              Bakery  0.04


----Barajas----
                venue  freq
0               Hotel  0.23
1  Spanish Restaurant  0.10
2          Restaurant  0.10
3         Coffee Shop  0.06
4    Tapas Restaurant  0.06


----Carabanchel----
                  venue  freq
0           Pizza Place  0.12
1  Fast Food Restaurant  0.12
2                 Plaza  0.12
3          Burger Joint  0.12
4          Soccer Field  0.12


----Centro----
                venue  freq
0    Tapas Restaurant  0.13
1  Spanish Restaurant  0.11
2               Plaza  0.07
3              Hostel  0.07
4              Bistro  0.05


----Chamartín----
                      venue  freq
0                Restaurant  0.21
1        Spanish Restaurant  0.08
2          Tapas Restaurant  0.08
3  Mediterranean Restaurant  0.04
4                     Plaza  0.04


----Chamberí--

###### A function to sort the venues in descending order.

In [146]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

###### Let's put data into a pandas dataframe with the top 10 venues for each district.

In [147]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Madrid_districts_venues_sorted = pd.DataFrame(columns=columns)
Madrid_districts_venues_sorted['District'] = Madrid_grouped['District']

for ind in np.arange(Madrid_grouped.shape[0]):
    Madrid_districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Madrid_grouped.iloc[ind, :], num_top_venues)

Madrid_districts_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arganzuela,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant,Gym / Fitness Center,Beer Garden,Hotel,Market,Brewery
1,Barajas,Hotel,Restaurant,Spanish Restaurant,Tapas Restaurant,Gastropub,Coffee Shop,Café,Wine Bar,Mexican Restaurant,Diner
2,Carabanchel,Soccer Field,Tapas Restaurant,Fast Food Restaurant,Pizza Place,Plaza,Burger Joint,Bakery,Nightclub,Flea Market,Farmers Market
3,Centro,Tapas Restaurant,Spanish Restaurant,Plaza,Hostel,Bistro,Cocktail Bar,Hotel,Wine Bar,Restaurant,Bar
4,Chamartín,Restaurant,Tapas Restaurant,Spanish Restaurant,Sushi Restaurant,Seafood Restaurant,Furniture / Home Store,Plaza,Big Box Store,Gastropub,Steakhouse


###### Run k-means to cluster the neighborhood into 5 clusters.

In [151]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Madrid_grouped_clustering = Madrid_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0])

In [162]:
# add clustering labels
Madrid_districts_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Madrid_merged = Final_MadridDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Madrid_merged = Madrid_merged.join(Madrid_districts_venues_sorted.set_index('District'), on='Name')

# Resulting Clusters labels are float. Converted to int because folium.CircleMarker need int values to color.
Madrid_merged['Cluster Labels'] = Madrid_merged['Cluster Labels'].astype(int)

Madrid_merged.head()

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Centro,522.82,"131,928",252.34,40.415347,-3.707371,0,Tapas Restaurant,Spanish Restaurant,Plaza,Hostel,Bistro,Cocktail Bar,Hotel,Wine Bar,Restaurant,Bar
1,2,Arganzuela,646.22,"151,965",235.16,40.402733,-3.695403,0,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant,Gym / Fitness Center,Beer Garden,Hotel,Market,Brewery
2,3,Retiro,546.62,"118,516",216.82,40.408072,-3.676729,0,Spanish Restaurant,Bakery,Grocery Store,Supermarket,Museum,Tapas Restaurant,Bar,Gym,Board Shop,Boutique
3,4,Salamanca,539.24,"143,800",266.67,40.429722,-3.679750,0,Spanish Restaurant,Boutique,Restaurant,Bakery,Mediterranean Restaurant,Tapas Restaurant,Coffee Shop,Mexican Restaurant,Hotel,Liquor Store
4,5,Chamartín,917.55,"143,424",156.31,40.462059,-3.676600,0,Restaurant,Tapas Restaurant,Spanish Restaurant,Sushi Restaurant,Seafood Restaurant,Furniture / Home Store,Plaza,Big Box Store,Gastropub,Steakhouse


###### Visualizing the resulting clusters in a map

In [164]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

# create map
Madrid_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map, 
markers_colors = []
for lat, lon, poi, cluster in zip(Madrid_merged['Latitude'], Madrid_merged['Longitude'], Madrid_merged['Name'], Madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],  
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Madrid_map_clusters)
       
Madrid_map_clusters

###### We repeat the same process only for the hotel category

In [166]:
#from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

Madrid_hotels_grouped_clustering = Madrid_hotels_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Madrid_hotels_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 1, 0, 1, 1, 0])

In [168]:
# add clustering labels
Madrid_districts_hotels = Madrid_hotels_grouped
Madrid_districts_hotels.insert(0, 'Cluster Labels', kmeans.labels_)

Madrid_hotels_merged = Final_MadridDF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Madrid_hotels_merged = Madrid_hotels_merged.join(Madrid_districts_hotels.set_index('District'), on='Name')

# Resulting Clusters labels are float. Converted to int because folium.CircleMarker need int values to color.
Madrid_hotels_merged['Cluster Labels'] = Madrid_hotels_merged['Cluster Labels'].astype(int)

Madrid_hotels_merged.head()

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Latitude,Longitude,Cluster Labels,Hotel
0,1,Centro,522.82,"131,928",252.34,40.415347,-3.707371,1,0.026667
1,2,Arganzuela,646.22,"151,965",235.16,40.402733,-3.695403,1,0.022472
2,3,Retiro,546.62,"118,516",216.82,40.408072,-3.676729,1,0.000000
3,4,Salamanca,539.24,"143,800",266.67,40.429722,-3.679750,1,0.021739
4,5,Chamartín,917.55,"143,424",156.31,40.462059,-3.676600,1,0.000000


In [169]:
#import matplotlib.cm as cm
#import matplotlib.colors as colors
# import k-means from clustering stage
#from sklearn.cluster import KMeans

# create map
Madrid_hotels_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map, 
markers_colors = []
for lat, lon, poi, cluster in zip(Madrid_hotels_merged['Latitude'], Madrid_hotels_merged['Longitude'], Madrid_hotels_merged['Name'], Madrid_hotels_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],  
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Madrid_hotels_map_clusters)
       
Madrid_hotels_map_clusters

###### Examine Cluster 1.

In [171]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 0, Madrid_merged.columns[[1] + list(range(5, Madrid_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,40.415347,-3.707371,0,Tapas Restaurant,Spanish Restaurant,Plaza,Hostel,Bistro,Cocktail Bar,Hotel,Wine Bar,Restaurant,Bar
1,Arganzuela,40.402733,-3.695403,0,Restaurant,Spanish Restaurant,Grocery Store,Bakery,Tapas Restaurant,Gym / Fitness Center,Beer Garden,Hotel,Market,Brewery
2,Retiro,40.408072,-3.676729,0,Spanish Restaurant,Bakery,Grocery Store,Supermarket,Museum,Tapas Restaurant,Bar,Gym,Board Shop,Boutique
3,Salamanca,40.429722,-3.679750,0,Spanish Restaurant,Boutique,Restaurant,Bakery,Mediterranean Restaurant,Tapas Restaurant,Coffee Shop,Mexican Restaurant,Hotel,Liquor Store
4,Chamartín,40.462059,-3.676600,0,Restaurant,Tapas Restaurant,Spanish Restaurant,Sushi Restaurant,Seafood Restaurant,Furniture / Home Store,Plaza,Big Box Store,Gastropub,Steakhouse
6,Chamberí,40.434040,-3.703790,0,Spanish Restaurant,Restaurant,Bar,Tapas Restaurant,Bakery,Café,Brewery,Coffee Shop,Italian Restaurant,Plaza
7,Fuencarral-El Pardo,40.478611,-3.709722,0,Clothing Store,Italian Restaurant,Fast Food Restaurant,Restaurant,Tapas Restaurant,Burger Joint,Pizza Place,Shopping Mall,Kebab Restaurant,Sandwich Place
8,Moncloa-Aravaca,40.435151,-3.718765,0,Spanish Restaurant,Restaurant,Japanese Restaurant,Pub,Ice Cream Shop,Bakery,Bar,Pizza Place,Mediterranean Restaurant,Burger Joint
9,Latina,40.411360,-3.708417,0,Tapas Restaurant,Spanish Restaurant,Bar,Plaza,Hotel,Vegetarian / Vegan Restaurant,Café,Restaurant,Mediterranean Restaurant,Bistro
12,Puente de Vallecas,40.398204,-3.669059,0,Fast Food Restaurant,Hotel,Tapas Restaurant,Gym,Grocery Store,Supermarket,Café,Breakfast Spot,Market,Pub


###### Examine Cluster 2.

In [172]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 1, Madrid_merged.columns[[1] + list(range(5, Madrid_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,San Blas-Canillejas,40.426001,-3.612764,1,Metro Station,Shopping Mall,Snack Place,Pizza Place,Gym,Grocery Store,Asian Restaurant,Supermarket,Gas Station,Electronics Store


###### Examine Cluster 3.

In [173]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 2, Madrid_merged.columns[[1] + list(range(5, Madrid_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Carabanchel,40.38367,-3.72799,2,Soccer Field,Tapas Restaurant,Fast Food Restaurant,Pizza Place,Plaza,Burger Joint,Bakery,Nightclub,Flea Market,Farmers Market


###### Examine Cluster 4.

In [174]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 3, Madrid_merged.columns[[1] + list(range(5, Madrid_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Villaverde,40.349998,-3.701143,3,Grocery Store,Spanish Restaurant,Mediterranean Restaurant,Train Station,Gastropub,Women's Store,Electronics Store,Flea Market,Fast Food Restaurant,Farmers Market


###### Examine Cluster 5.

In [175]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 4, Madrid_merged.columns[[1] + list(range(5, Madrid_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Tetuán,40.459751,-3.697500,4,Spanish Restaurant,Restaurant,Hotel,Brazilian Restaurant,Grocery Store,Supermarket,Brewery,Chinese Restaurant,Coffee Shop,Pizza Place
11,Usera,40.381389,-3.706944,4,Seafood Restaurant,Spanish Restaurant,Chinese Restaurant,Grocery Store,Bubble Tea Shop,Fast Food Restaurant,Café,Theater,Noodle House,Asian Restaurant
14,Ciudad Lineal,40.445049,-3.651320,4,Spanish Restaurant,Grocery Store,Hotel,Italian Restaurant,Coffee Shop,Pizza Place,Bakery,Office,Chinese Restaurant,Restaurant
17,Villa de Vallecas,40.379597,-3.621347,4,Bar,Plaza,Spanish Restaurant,Church,Grocery Store,Soccer Field,Park,Dessert Shop,Fabric Shop,Food


###### Examine Cluster 1. Only Hotels

In [170]:
Madrid_hotels_merged.loc[Madrid_hotels_merged['Cluster Labels'] == 0, Madrid_hotels_merged.columns[[1] + list(range(5, Madrid_hotels_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,Hotel
5,Tetuán,40.459751,-3.697500,0,0.048780
9,Latina,40.411360,-3.708417,0,0.060000
12,Puente de Vallecas,40.398204,-3.669059,0,0.071429
14,Ciudad Lineal,40.445049,-3.651320,0,0.064516


###### Examine Cluster 2. Only Hotels

In [176]:
Madrid_hotels_merged.loc[Madrid_hotels_merged['Cluster Labels'] == 1, Madrid_hotels_merged.columns[[1] + list(range(5, Madrid_hotels_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,Hotel
0,Centro,40.415347,-3.707371,1,0.026667
1,Arganzuela,40.402733,-3.695403,1,0.022472
2,Retiro,40.408072,-3.676729,1,0.000000
3,Salamanca,40.429722,-3.679750,1,0.021739
4,Chamartín,40.462059,-3.676600,1,0.000000
6,Chamberí,40.434040,-3.703790,1,0.010000
7,Fuencarral-El Pardo,40.478611,-3.709722,1,0.000000
8,Moncloa-Aravaca,40.435151,-3.718765,1,0.017857
10,Carabanchel,40.383670,-3.727990,1,0.000000
11,Usera,40.381389,-3.706944,1,0.000000


###### Examine Cluster 3. Only Hotels

In [177]:
Madrid_hotels_merged.loc[Madrid_hotels_merged['Cluster Labels'] == 2, Madrid_hotels_merged.columns[[1] + list(range(5, Madrid_hotels_merged.shape[1]))]]

,Name,Latitude,Longitude,Cluster Labels,Hotel
20,Barajas,40.470278,-3.585,2,0.225806


## 4.Results 

Based on the information provided by Foursquare, we can see how most of the districts of Madrid are well provided with restaurants and other venues that can attract the attention of tourists.

The Barajas district is the one with the highest concentration of hotels. This makes sense, because it is the district where the international airport is located. Therefore this district should be avoided. It is also located on the outskirts of town.

Among the most central districts we can highlight two of them. Salamanca and Retiro.

Salamanca is well known for being one of the wealthiest and expensive areas in Madrid, with a high living cost and one of the highest real estate prices in the city. Serrano street is listed as the third most expensive street in Spain.
A significant number of diplomatic missions are set in Salamanca district, such as the Swiss embassy at Nuñez de Balboa with Goya street, the French embassy at Villalar Street, the United States embassy at 74 Serrano street, or the Italian embassy in Juan Bravo Street.(wikipedia)

Retiro's history has been strongly determined by the presence of the park with the same name and the railway facilities located at the south and west of this District.
Retiro Park, together with Paseo del Prado, has given its own personality to the area of Los Jerónimos, while the rest of the District is determined by Atocha Railway Station and previously also by Niño Jesús Station.
During the 20th Century, the area consolidated as residential areas for people with a growing economic power.(wikipedia)

Both districts are well-stocked with tourist resources and have a low concentration of hotels. The biggest drawback is finding soil to build a hotel. Perhaps it would be easier to find a building to reform as a hotel.

## 5.Conclusion

Based on the analyzed data, my advice would be to focus on only two Madrid districts, Salamanca and Retiro, when investing on a new hotel. Especially being a high standing or luxury hotel.